In [2]:
# import libraries
import numpy as np
import pandas as pd
import io
import requests
from datetime import datetime, timezone, timedelta
import datetime as dt
import time
from collections import Counter
from PIL import Image

import six
from six import StringIO
import sys
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

# import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# import seaborn
import seaborn as sns

# import plotly graph objects
import plotly.graph_objs as go
import plotly.express as px

# import scipy.cluster.hierarchy
import scipy
import scipy.linalg as la
import scipy.cluster.hierarchy as sch
from scipy.cluster.vq import whiten, kmeans, vq
from scipy.stats import multivariate_normal
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

# import category encoders
import category_encoders as ce

import statsmodels.api as sm

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
import sklearn.model_selection as model_selection
import sklearn.model_selection as cross_validation
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import average_precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_precision_recall_curve, plot_confusion_matrix
from sklearn import linear_model, datasets
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

#for showing 2D plot
%matplotlib inline 

# to be able to see multiple ouputs from sungle cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Step 1. Import sample data

In [25]:
# import sample data - flights
file_path = '../data/flights.csv'
flights = pd.read_csv(file_path)
print('sample data: flights')
flights.info()
flights.head()

file_path2 = '../data/flights_test.csv'
flights_test = pd.read_csv(file_path2)
print('sample data: flights_test')
flights_test.info()
flights_test.head()

file_path3 = '../data/fuel_consumption.csv'
fuel_cons = pd.read_csv(file_path3)
print('sample data: fuel_consumption')
fuel_cons.info()
fuel_cons.head()

file_path4 = '../data/passengers.csv'
passengers = pd.read_csv(file_path4)
print('sample data: passengers')
passengers.info()
passengers.head()

sample data: flights
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fl_date              1000 non-null   object 
 1   mkt_unique_carrier   1000 non-null   object 
 2   branded_code_share   1000 non-null   object 
 3   mkt_carrier          1000 non-null   object 
 4   mkt_carrier_fl_num   1000 non-null   int64  
 5   op_unique_carrier    1000 non-null   object 
 6   tail_num             1000 non-null   object 
 7   op_carrier_fl_num    1000 non-null   int64  
 8   origin_airport_id    1000 non-null   int64  
 9   origin               1000 non-null   object 
 10  origin_city_name     1000 non-null   object 
 11  dest_airport_id      1000 non-null   int64  
 12  dest                 1000 non-null   object 
 13  dest_city_name       1000 non-null   object 
 14  crs_dep_time         1000 non-null   int64  
 15  dep_time          

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-12-25,AA,AA_CODESHARE,AA,5178,OH,N609NN,5178,11057,CLT,...,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-25,AA,AA_CODESHARE,AA,5178,OH,N609NN,5178,11996,GSP,...,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-12-25,AA,AA_CODESHARE,AA,5180,OH,N527EA,5180,11057,CLT,...,564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-25,AA,AA_CODESHARE,AA,5180,OH,N527EA,5180,12197,HPN,...,564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-25,AA,AA_CODESHARE,AA,5182,OH,N548NN,5182,11267,DAY,...,370,2.0,0.0,1.0,0.0,64.0,NaN,NaN,NaN,NaN


sample data: flights_test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   fl_date             1000 non-null   int64 
 1   mkt_unique_carrier  1000 non-null   object
 2   branded_code_share  1000 non-null   object
 3   mkt_carrier         1000 non-null   object
 4   mkt_carrier_fl_num  1000 non-null   int64 
 5   op_unique_carrier   1000 non-null   object
 6   tail_num            1000 non-null   object
 7   op_carrier_fl_num   1000 non-null   int64 
 8   origin_airport_id   1000 non-null   int64 
 9   origin              1000 non-null   object
 10  origin_city_name    1000 non-null   object
 11  dest_airport_id     1000 non-null   int64 
 12  dest                1000 non-null   object
 13  dest_city_name      1000 non-null   object
 14  crs_dep_time        1000 non-null   int64 
 15  crs_arr_time        1000 non-null   int64 
 16 

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,1577865600000,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,1577865600000,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,1577865600000,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,1577865600000,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,1577865600000,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


sample data: fuel_consumption
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              1000 non-null   int64  
 1   airline_id         999 non-null    float64
 2   unique_carrier     999 non-null    object 
 3   carrier            1000 non-null   object 
 4   carrier_name       999 non-null    object 
 5   carrier_group_new  1000 non-null   int64  
 6   sdomt_gallons      1000 non-null   int64  
 7   satl_gallons       1000 non-null   int64  
 8   spac_gallons       1000 non-null   int64  
 9   slat_gallons       1000 non-null   int64  
 10  sint_gallons       1000 non-null   int64  
 11  ts_gallons         1000 non-null   int64  
 12  tdomt_gallons      1000 non-null   int64  
 13  tint_gallons       1000 non-null   int64  
 14  total_gallons      1000 non-null   int64  
 15  sdomt_cost         1000 non-null   int64  


,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,NaN,0JQ,NaN,1,0,0,0,0,...,0,0,0,0,0,0,5100,0,5100,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0,0,0,0,...,0,0,0,0,0,0,396216,140239,536455,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848,0,0,0,...,522405,0,0,0,0,522405,569497,0,569497,2016


sample data: passengers
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   departures_scheduled   1000 non-null   int64 
 1   departures_performed   1000 non-null   int64 
 2   payload                1000 non-null   int64 
 3   seats                  1000 non-null   int64 
 4   passengers             1000 non-null   int64 
 5   freight                1000 non-null   int64 
 6   mail                   1000 non-null   int64 
 7   distance               1000 non-null   int64 
 8   ramp_to_ramp           1000 non-null   int64 
 9   air_time               1000 non-null   int64 
 10  unique_carrier         1000 non-null   object
 11  airline_id             1000 non-null   int64 
 12  unique_carrier_name    1000 non-null   object
 13  region                 1000 non-null   object
 14  carrier                1000 non-null   object
 15

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0,1,3300,9,0,0,0,18,19,13,...,US,United States,4,416,1,2015,5,1,L,DU
1,0,2,11200,0,0,3309,2325,21,35,23,...,US,United States,4,412,2,2015,5,1,G,DU
2,0,1,5600,0,0,0,1416,51,28,22,...,US,United States,4,412,2,2015,5,1,G,DU
3,0,2,11200,0,0,0,0,109,96,84,...,US,United States,4,412,2,2015,5,1,G,DU
4,0,3,16800,0,0,589,3666,71,97,79,...,US,United States,4,412,2,2015,5,1,G,DU


### Step 2. Draw Database Diagram

In [26]:
# function that return db table from data dataframe:
def get_dbtable(df):
    colname = df.columns.tolist()
    non_na = df.count()
    nullcount = df.isnull().sum()
    dtypes = df.dtypes

    df_table = pd.DataFrame({'columns': colname, 
                                  'Non-Null Count': non_na,
                                  'Null Count': nullcount,
                                  'Dtype': dtypes})

    # remove own index with default index
    df_table.reset_index(inplace = True, drop = True)
    
    return df_table

In [27]:
fl_table = get_dbtable(flights)
fl_test_table = get_dbtable(flights_test)
fuel_table = get_dbtable(fuel_cons)
passengr_table = get_dbtable(passengers)

fl_table
fl_test_table
fuel_table
passengr_table

,columns,Non-Null Count,Null Count,Dtype
0,fl_date,1000,0,object
1,mkt_unique_carrier,1000,0,object
2,branded_code_share,1000,0,object
3,mkt_carrier,1000,0,object
4,mkt_carrier_fl_num,1000,0,int64
5,op_unique_carrier,1000,0,object
6,tail_num,1000,0,object
7,op_carrier_fl_num,1000,0,int64
8,origin_airport_id,1000,0,int64
9,origin,1000,0,object


,columns,Non-Null Count,Null Count,Dtype
0,fl_date,1000,0,int64
1,mkt_unique_carrier,1000,0,object
2,branded_code_share,1000,0,object
3,mkt_carrier,1000,0,object
4,mkt_carrier_fl_num,1000,0,int64
5,op_unique_carrier,1000,0,object
6,tail_num,1000,0,object
7,op_carrier_fl_num,1000,0,int64
8,origin_airport_id,1000,0,int64
9,origin,1000,0,object


,columns,Non-Null Count,Null Count,Dtype
0,month,1000,0,int64
1,airline_id,999,1,float64
2,unique_carrier,999,1,object
3,carrier,1000,0,object
4,carrier_name,999,1,object
5,carrier_group_new,1000,0,int64
6,sdomt_gallons,1000,0,int64
7,satl_gallons,1000,0,int64
8,spac_gallons,1000,0,int64
9,slat_gallons,1000,0,int64


,columns,Non-Null Count,Null Count,Dtype
0,departures_scheduled,1000,0,int64
1,departures_performed,1000,0,int64
2,payload,1000,0,int64
3,seats,1000,0,int64
4,passengers,1000,0,int64
5,freight,1000,0,int64
6,mail,1000,0,int64
7,distance,1000,0,int64
8,ramp_to_ramp,1000,0,int64
9,air_time,1000,0,int64


In [28]:
flights['flights'].value_counts()

1    1000
Name: flights, dtype: int64

In [30]:
flights['arr_delay']
flights['dep_delay']

0       1.0
1     -29.0
2       4.0
3     -16.0
4      67.0
       ... 
995    18.0
996   -26.0
997     7.0
998   -22.0
999    26.0
Name: arr_delay, Length: 1000, dtype: float64

0       0.0
1      -3.0
2      -1.0
3       3.0
4      66.0
       ... 
995    13.0
996    -3.0
997    20.0
998    -4.0
999    20.0
Name: dep_delay, Length: 1000, dtype: float64